In [1]:
import sys
sys.path.insert(0, "../..")
import config as cfg
import gc
import os
from tqdm.notebook import tqdm
from helper import check_path, seed_everything
from collections import defaultdict

In [2]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [5]:
EXPERIMENT_FAMILY_NAME = 'keras'
EXPERIMENT_NAME = 'labse_emb'
N_RANDOM_SEEDS = 1
N_SPLITS = 5
N_EPOCHS = 100
BATCH_SIZE = 32

In [6]:
RANDOM_STATE = 77
seed_everything(RANDOM_STATE)

In [7]:
def filter_rare_categories(df: pd.DataFrame) -> pd.DataFrame:
    rare_categories = [12]
    return df.loc[~df[cfg.TARGET].isin(rare_categories)]

In [8]:

train = pd.read_csv(cfg.ORIG_TRAIN_PATH, usecols=[cfg.TARGET, cfg.ID_COL, cfg.TEXT_COL]).set_index('id')  #  cfg.THEME_COL
test = pd.read_csv(cfg.ORIG_TEST_PATH, usecols=[cfg.ID_COL, cfg.TEXT_COL]).set_index('id')

train = train.rename(columns={cfg.TEXT_COL: 'text'})
test = test.rename(columns={cfg.TEXT_COL: 'text'})

CLASSES = np.sort(train[cfg.TARGET].unique()).tolist()

# train = filter_rare_categories(train)

In [9]:
# EMB_NAME = 'smaller_LaBSE_15lang'
# train_emb = pd.read_pickle(os.path.join(cfg.DATA_PATH, EMB_NAME, 'train.pkl'))
# test_emb = pd.read_pickle(os.path.join(cfg.DATA_PATH, EMB_NAME, 'test.pkl'))

# train = train.join(train_emb)
# test = test.join(test_emb)

# del train_emb, test_emb; gc.collect()

In [10]:
X_train, y_train = train.drop(cfg.TARGET, axis=1), train[cfg.TARGET]
y_train_onecol = y_train.copy()
y_train = pd.get_dummies(y_train)
assert y_train.shape[1] == len(CLASSES)

In [11]:
pred_proba_oof = pd.DataFrame(data=np.zeros(shape=(len(train), len(CLASSES))), index=train.index, columns=CLASSES)
pred_proba_test = pd.DataFrame(data=np.zeros(shape=(len(test), len(CLASSES))), index=test.index, columns=CLASSES)

In [12]:
from typing import Optional


def df_to_dataset(data: pd.DataFrame, labels: Optional[pd.DataFrame]=None, shuffle: bool=True, batch_size: int=32):
  df = data.copy()
  df = {key: value.values[:, tf.newaxis] for key, value in data.items()}
  
  if labels is None:
    ds = tf.data.Dataset.from_tensor_slices((dict(df),))
  else:
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(data))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds


def get_keras_input(
    data: pd.DataFrame,
    labels: Optional[pd.DataFrame] = None,
    shuffle=False
    ) -> tf.data.Dataset:
    return df_to_dataset(
        data=data,
        labels=labels,
        shuffle=shuffle, 
        batch_size=BATCH_SIZE)
        


In [13]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer("https://tfhub.dev/jeongukjae/smaller_LaBSE_15lang_preprocess/1", name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer("https://tfhub.dev/jeongukjae/smaller_LaBSE_15lang/1", trainable=False, name='encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(len(CLASSES), activation=None, name='classifier')(net)
    model = tf.keras.Model(text_input, net)

    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    metrics = tf.keras.metrics.AUC(from_logits=True, name='auc')

    epochs = N_EPOCHS
    n = (len(train) // N_SPLITS) * (N_SPLITS-1)
    steps_per_epoch = n // BATCH_SIZE
    num_train_steps = steps_per_epoch * epochs
    num_warmup_steps = int(0.1*num_train_steps)

    init_lr = 3e-5
    optimizer = optimization.create_optimizer(init_lr=init_lr,
                                            num_train_steps=num_train_steps,
                                            num_warmup_steps=num_warmup_steps,
                                            optimizer_type='adamw')

    model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
    return model
    

In [14]:
cv = StratifiedKFold(n_splits=cfg.N_SPLITS, random_state=cfg.RANDOM_STATE, shuffle=True)

test_pool = get_keras_input(
        data=test,
        shuffle=False
        )


metrics = defaultdict(list)
fold = 0
for train_idx, val_idx in tqdm(cv.split(X_train, y_train_onecol), total=cfg.N_SPLITS):

    train_pool = get_keras_input(
        data=X_train.iloc[train_idx], 
        labels=y_train.iloc[train_idx],
        shuffle=True
        )

    val_pool = get_keras_input(
        data=X_train.iloc[val_idx], 
        labels=y_train.iloc[val_idx],
        shuffle=False
        )
        
    for random_seed in tqdm(range(N_RANDOM_SEEDS), total=N_RANDOM_SEEDS):

        clf = build_classifier_model()
        
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=3, mode='max', restore_best_weights=True)
        clf.fit(x=train_pool, validation_data=val_pool, epochs=N_EPOCHS, callbacks=[early_stopping])
        
        model_name = f'{EXPERIMENT_NAME}_fold_{fold}_rs_{random_seed}'
        model_path = os.path.join(cfg.MODELS_PATH, EXPERIMENT_FAMILY_NAME, EXPERIMENT_NAME)
        check_path(model_path)
        clf.save(os.path.join(model_path, model_name), include_optimizer=False)
        
        pred_proba_oof_val = tf.nn.softmax(clf.predict(val_pool)).numpy()
        pred_proba_oof_train = tf.nn.softmax(clf.predict(train_pool)).numpy()

        pred_proba_oof.iloc[val_idx, :] += pred_proba_oof_val
        pred_proba_test.iloc[:, :] += tf.nn.softmax(clf.predict(test_pool)).numpy()

        y_val_oof = y_train_onecol.iloc[val_idx]

        val_auc = roc_auc_score(y_val_oof, pred_proba_oof_val , multi_class='ovo', labels=CLASSES)
        metrics['val_auc'].append(val_auc)
        print('val auc', val_auc)
        
        del clf; gc.collect()
    del train_pool,val_pool; gc.collect() 
        
    fold += 1
pred_proba_oof /= N_RANDOM_SEEDS
pred_proba_test /= (cfg.N_SPLITS * N_RANDOM_SEEDS)

2022-07-21 21:18:50.049906: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-21 21:18:50.085586: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-21 21:18:50.085792: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-21 21:18:50.087167: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

  0%|          | 0/5 [00:00<?, ?it/s]

/home/as/miniconda3/envs/tf/lib/python3.9/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/100
50/50 [==============================] - 43s 692ms/step - loss: 2.8990 - auc: 0.5522 - val_loss: 2.8489 - val_auc: 0.5781
Epoch 2/100
50/50 [==============================] - 32s 653ms/step - loss: 2.8844 - auc: 0.5608 - val_loss: 2.8116 - val_auc: 0.5998
Epoch 3/100
50/50 [==============================] - 32s 652ms/step - loss: 2.8310 - auc: 0.5899 - val_loss: 2.7506 - val_auc: 0.6350
Epoch 4/100
50/50 [==============================] - 32s 652ms/step - loss: 2.7631 - auc: 0.6261 - val_loss: 2.6679 - val_auc: 0.6805
Epoch 5/100
50/50 [==============================] - 33s 654ms/step - loss: 2.6616 - auc: 0.6789 - val_loss: 2.5666 - val_auc: 0.7317
Epoch 6/100
50/50 [==============================] - 32s 653ms/step - loss: 2.5615 - auc: 0.7254 - val_loss: 2.4502 - val_auc: 0.7810
Epoch 7/100
50/50 [==============================] - 32s 653ms/step - loss: 2.4351 - auc: 0.7772 - val_loss: 2.3228 - val_auc: 0.8227
Epoch 8/100
50/50 [==============================] - 33s 654ms

...

Epoch 1/100
50/50 [==============================] - 42s 692ms/step - loss: 3.1308 - auc: 0.4299 - val_loss: 3.1376 - val_auc: 0.4108
Epoch 2/100
50/50 [==============================] - 33s 654ms/step - loss: 3.0967 - auc: 0.4461 - val_loss: 3.1000 - val_auc: 0.4291
Epoch 3/100
50/50 [==============================] - 32s 639ms/step - loss: 3.0488 - auc: 0.4672 - val_loss: 3.0383 - val_auc: 0.4600
Epoch 4/100
50/50 [==============================] - 32s 634ms/step - loss: 2.9750 - auc: 0.5071 - val_loss: 2.9547 - val_auc: 0.5046
Epoch 5/100
50/50 [==============================] - 32s 640ms/step - loss: 2.8693 - auc: 0.5584 - val_loss: 2.8518 - val_auc: 0.5629
Epoch 6/100
50/50 [==============================] - 32s 635ms/step - loss: 2.7627 - auc: 0.6152 - val_loss: 2.7329 - val_auc: 0.6314
Epoch 7/100
50/50 [==============================] - 32s 637ms/step - loss: 2.6270 - auc: 0.6874 - val_loss: 2.6018 - val_auc: 0.7043
Epoch 8/100
50/50 [==============================] - 32s 639ms

32/32 [==============================] - 18s 530ms/step
train auc 0.4921892466262901 val auc 0.7076036590076551


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/100
50/50 [==============================] - 44s 697ms/step - loss: 3.1758 - auc: 0.4241 - val_loss: 3.1275 - val_auc: 0.4294
Epoch 2/100
50/50 [==============================] - 34s 686ms/step - loss: 3.1457 - auc: 0.4365 - val_loss: 3.0882 - val_auc: 0.4474
Epoch 3/100
50/50 [==============================] - 34s 686ms/step - loss: 3.0990 - auc: 0.4599 - val_loss: 3.0232 - val_auc: 0.4793
Epoch 4/100
50/50 [==============================] - 34s 688ms/step - loss: 3.0092 - auc: 0.5001 - val_loss: 2.9345 - val_auc: 0.5238
Epoch 5/100
50/50 [==============================] - 34s 688ms/step - loss: 2.9123 - auc: 0.5465 - val_loss: 2.8233 - val_auc: 0.5830
Epoch 6/100
50/50 [==============================] - 34s 688ms/step - loss: 2.7820 - auc: 0.6114 - val_loss: 2.6937 - val_auc: 0.6525
Epoch 7/100
50/50 [==============================] - 34s 689ms/step - loss: 2.6404 - auc: 0.6816 - val_loss: 2.5492 - val_auc: 0.7258
Epoch 8/100
50/50 [==============================] - 34s 689ms

32/32 [==============================] - 19s 549ms/step
train auc 0.49084757444453847 val auc 0.7530983544550861


In [15]:
oof_auc_score = roc_auc_score(y_train, pred_proba_oof , multi_class='ovo', labels=CLASSES)
print('oof_auc_score', oof_auc_score)

oof_auc_score 0.7620819837965503


In [ ]:
# 0.7620819837965503

In [16]:
leak_test = pd.read_pickle(os.path.join(cfg.DATA_PATH, 'test_leak.pkl'))
notnull = leak_test.notnull()
leak_test_auc_score = roc_auc_score(leak_test.loc[notnull], pred_proba_test.loc[notnull], multi_class='ovo', labels=CLASSES)
print('leak_test_auc_score', leak_test_auc_score)


leak_test_auc_score 0.7887553869552576


In [17]:
# 0.8399045223216896

In [18]:
submission = pd.read_csv(cfg.SAMPLE_SUBMIT_PATH).set_index('id')
assert submission.index.equals(pred_proba_test.index)
submission[cfg.TARGET] = pred_proba_test.idxmax(1)

submission_path = os.path.join(cfg.SUBMISSION_PATH, EXPERIMENT_FAMILY_NAME)
check_path(submission_path)
submission.to_csv(os.path.join(submission_path, f'{EXPERIMENT_NAME}.csv'))

pred_proba_oof_path = os.path.join(cfg.OOF_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(pred_proba_oof_path)
pred_proba_oof.to_pickle(os.path.join(pred_proba_oof_path, f'{EXPERIMENT_NAME}.pkl'))

pred_proba_test_path = os.path.join(cfg.TEST_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(pred_proba_test_path)
pred_proba_test.to_pickle(os.path.join(pred_proba_test_path, f'{EXPERIMENT_NAME}.pkl'))